In [2]:
import pandas as pd
import seaborn as sns


In [3]:
wsb = pd.read_pickle('../may_wsb.pkl')

In [4]:
wsb

,title,id,created_utc,author_fullname,is_self,score,upvote_ratio,total_awards_received,selftext,url
0,Glorious Oracle of Ohama achieves pareto effic...,n6lg1w,1620345568,t2_3qdkzqjm,False,1,0.99,0,,https://i.redd.it/mspkkklr9lx61.png
1,FLGT TO THE MOON...1ST QTR #S TODAY,n6lfdp,1620345502,t2_a0mf559i,True,1,1.00,0,FLGT TO THE MOON...1ST QTR #S TODAY,https://www.reddit.com/r/wallstreetbets/commen...
2,OCGN Watch for Tomorrow...,n6lfa5,1620345492,t2_a368t9uc,True,1,1.00,0,[removed],https://www.reddit.com/r/wallstreetbets/commen...
3,i think this fits here,n6lefj,1620345410,t2_4ac5c7gb,False,3,0.72,1,,https://www.youtube.com/watch?v=6aZ0JYy2IIc
4,Sad 😔,n6lecl,1620345402,t2_5qd02awb,False,36,0.86,0,,https://i.redd.it/ciguyky99lx61.jpg
...,...,...,...,...,...,...,...,...,...,...
9401,Lollllllllll,n26sjv,1619827492,t2_8jwdn4sp,False,1,1.00,0,,https://i.redd.it/uk59khg9hew61.jpg
9402,Patience is a virtue. Took awhile before Tesla...,n26rtq,1619827429,t2_6ny07hwq,False,2,1.00,0,,https://v.redd.it/touz8edygew61
9403,First 500 to sign up receive 100k DaBaby,n26r4b,1619827368,t2_a1z5kokh,False,0,0.23,1,,https://docs.google.com/forms/d/e/1FAIpQLSeKQw...
9404,"If there is one thing I have learnt on r/wsb, ...",n26qr6,1619827339,t2_6b8tzrby,True,1,1.00,0,[removed],https://www.reddit.com/r/wallstreetbets/commen...
